In [5]:
#Run with Python 3

import pandas as pd
import numpy as np
import urllib.request
import shutil
import re
import os
import pathlib
import argparse
import pdb
import threading
from bs4 import BeautifulSoup

def site_open(site):
    '''Makes connection and opens up target website. Returns a website object.'''
    try:
        #sets up request object
        req = urllib.request.Request(site)

        #adds User-Agent info to request object
        req.add_header("User-Agent","Mozilla/5.0 (Macintosh; Intel Mac OS X 10_6_8) \
         AppleWebKit/536.5 (KHTML, like Gecko) Chrome/19.0.1084.46 Safari/536.5")

        #opens up site
        website = urllib.request.urlopen(req)

        return website
    except urllib.request.URLError:
#            print('Could not connect to '+ site + '!')
           pass
    
def soup_site(site):
    '''opens site and turns it into a format to easily parse the DOM. Returns a Soup Object'''
    try:
        website = site_open(site)
#         return BeautifulSoup(website, "html5lib")
        return BeautifulSoup(website, "lxml")
    except Exception as e:
        print(e)
        return -1

In [6]:
def get_web_data_from_link(link):
    return soup_site(link)

In [7]:
links = "http://hanvota.com/nhac/web/";
links = "http://1506.unknownsecret.info/mp3/Michael+Learns+to+Rock/Take+Me+to+Your+Heart/"
links = "http://www.dna.com.vn/folder_news/"
links = 'http://kinh.kinhphat.org/kinhgiang/NhatHanh/TrinhCongSon/'
print ("Geting infomation from: " + links)

#file_filter = [".jpg", ".gif", ".jpg"]
file_filter = [".mp3"]

soup = get_web_data_from_link(links);
#print (soup)

all_info = []
file_name = ''#
path = ''


if soup != -1:
    hyperlink_tags = soup.find_all('a')
    
    for hyperlink_tag in hyperlink_tags:
        #print (hyperlink_tag)
        hyperlink = hyperlink_tag.get('href') 
        hyperlink_value = hyperlink_tag.text
        
        #print (hyperlink)
        for f_filter in file_filter:
            if hyperlink.find(f_filter) != -1:      # keep all hyperlinks which contain pattern
                #print ("hyperlink:" + hyperlink)
                downlink = links + hyperlink
                if hyperlink.find('/') != -1:     # if hyperlink contains /  then check duplicate in path
                    path = hyperlink.rsplit('/', 1)[0]            # get the first part from last /
                    #print("Path:" + path)
                    if links.find(path) != -1:      # found overlap between links and hyperlink
                                                    # then remove overlap
                        downlink = downlink.replace(path, '',1)
                #print (downlink)

                if hyperlink_value.find(f_filter) != -1:  # if filename already provided, use it
                    file_name = hyperlink_value
                    #print("test: " + test)
                else:                                        # else create new 1 from the hyperlink
                    if hyperlink.find('/') != -1:     # if hyperlink contains /  then get the last field (name)
                        file_name = hyperlink.rsplit('/', 1)[1]
                    else:                             # else file name is same hyperlink
                        file_name = hyperlink     
                all_info.append([downlink, file_name])
for item in all_info:
    print (item)
#print (all_info)

Geting infomation from: http://kinh.kinhphat.org/kinhgiang/NhatHanh/TrinhCongSon/
['http://kinh.kinhphat.org/kinhgiang/NhatHanh/TrinhCongSon/TrinhCongSon-1.mp3', 'TrinhCongSon-1.mp3']
['http://kinh.kinhphat.org/kinhgiang/NhatHanh/TrinhCongSon/TrinhCongSon-2.mp3', 'TrinhCongSon-2.mp3']
['http://kinh.kinhphat.org/kinhgiang/NhatHanh/TrinhCongSon/TrinhCongSon-3.mp3', 'TrinhCongSon-3.mp3']


In [9]:
import urllib.request
import os
download_path = 'download\\'
# mode 
mode = 0o666
# check if download folder is exist
isdir = os.path.isdir(download_path)
if  not isdir:  # if not create new one
    os.mkdir(download_path, mode) 

#for item in all_info:
#    print('Downloading: ' + item[0])
#    urllib.request.urlretrieve(item[0], "download\\" +item[1])

for line_link in all_links:
    if line_link[-1] != '/':   # just download files, skip folder
        temp_name = line_link.replace('//','/')
        temp_name = temp_name.replace('http:/hdgsnn.gov.vn/files/','')
        temp_name = temp_name.replace('/','_')
        #print('Downloading: ' + line_link + " save name: " + temp_name)
        urllib.request.urlretrieve(line_link, download_path + temp_name)
    else:
        print ("skip: " + line_link)

NameError: name 'all_links' is not defined

In [53]:
# write to csv file
import csv
f = open("all_links.csv", 'w', newline='', encoding="utf-8")
writer = csv.writer(f, delimiter=",")
writer.writerow(["link", "file name"])
for item in all_info:
    writer.writerow(item)
f.close()

In [3]:
# this function will list all links as posible of a given path
link = 'http://kinh.kinhphat.org/kinhgiang/NhatHanh/'
#link = "http://1506.unknownsecret.info/mp3/Michael+Learns+to+Rock/"
print ("Geting infomation from: " + link)

all_links = []
all_links.append(link)
for cur_link in all_links:
    # check if current link is a file. 
    if cur_link[-1] == '/':   # process if current link is folder, skip if it's a file
        soup = get_web_data_from_link(cur_link);
        if soup != -1:  # handle if data is responded
            hyperlink_tags = soup.find_all('a')
            for hyperlinks in hyperlink_tags:
                hyperlink = hyperlinks.get('href')
                downlink = cur_link + hyperlink
                if hyperlink.find('/') != -1:     # if hyperlink contains /  then check duplicate in path
                    
                    path = hyperlink.rsplit('/', 1)[0]            # get the first part from last /
                    print ("hyperlink: " + hyperlink)
                    print ("path: " + path)
                    print ("cur_link: " + cur_link)
                    
                    if cur_link.find(path) != -1:      # found overlap between links and hyperlink
                                                    # then remove overlap
                        print ("hyperlxxxxink.find replace")
                        downlink = downlink.replace(path, '',1)
                print (downlink)
                if downlink not in all_links: # not exist in list, then add into it
                    # skip if its parrent folder
                    #print("found: " + hyperlink)
                    if hyperlink.find('../') == -1:
                        all_links.append(downlink)
                        #print ("Added: " + hyperlink)

Geting infomation from: http://kinh.kinhphat.org/kinhgiang/NhatHanh/
hyperlink: ../
path: ..
cur_link: http://kinh.kinhphat.org/kinhgiang/NhatHanh/
http://kinh.kinhphat.org/kinhgiang/NhatHanh/../
hyperlink: 2003/
path: 2003
cur_link: http://kinh.kinhphat.org/kinhgiang/NhatHanh/
http://kinh.kinhphat.org/kinhgiang/NhatHanh/2003/
hyperlink: 2004/
path: 2004
cur_link: http://kinh.kinhphat.org/kinhgiang/NhatHanh/
http://kinh.kinhphat.org/kinhgiang/NhatHanh/2004/
hyperlink: 2005-HaNoi/
path: 2005-HaNoi
cur_link: http://kinh.kinhphat.org/kinhgiang/NhatHanh/
http://kinh.kinhphat.org/kinhgiang/NhatHanh/2005-HaNoi/
hyperlink: 2007/
path: 2007
cur_link: http://kinh.kinhphat.org/kinhgiang/NhatHanh/
http://kinh.kinhphat.org/kinhgiang/NhatHanh/2007/
hyperlink: BonCauThanChu/
path: BonCauThanChu
cur_link: http://kinh.kinhphat.org/kinhgiang/NhatHanh/
http://kinh.kinhphat.org/kinhgiang/NhatHanh/BonCauThanChu/
hyperlink: BonLoaiXucThuc/
path: BonLoaiXucThuc
cur_link: http://kinh.kinhphat.org/kinhgiang/N

hyperlink: ../
path: ..
cur_link: http://kinh.kinhphat.org/kinhgiang/NhatHanh/2007/
http://kinh.kinhphat.org/kinhgiang/NhatHanh/2007/../
http://kinh.kinhphat.org/kinhgiang/NhatHanh/2007/02-22-07-BiQuyetAnLacTrongCuocSong-01.mp3
http://kinh.kinhphat.org/kinhgiang/NhatHanh/2007/02-22-07-BiQuyetAnLacTrongCuocSong-02.mp3
http://kinh.kinhphat.org/kinhgiang/NhatHanh/2007/02-23-07-NgheThuatThuongYeu-01.mp3
http://kinh.kinhphat.org/kinhgiang/NhatHanh/2007/02-23-07-NgheThuatThuongYeu-02.mp3
http://kinh.kinhphat.org/kinhgiang/NhatHanh/2007/02-26-07-TaiLapLaiTruyenThongChuaLSDaLat-01.mp3
http://kinh.kinhphat.org/kinhgiang/NhatHanh/2007/02-26-07-TaiLapLaiTruyenThongChuaLSDaLat-02.mp3
http://kinh.kinhphat.org/kinhgiang/NhatHanh/2007/03-01-07-NgoiGiuaGioXuan-01.mp3
http://kinh.kinhphat.org/kinhgiang/NhatHanh/2007/03-01-07-NgoiGiuaGioXuan-02.mp3
http://kinh.kinhphat.org/kinhgiang/NhatHanh/2007/03-02-07-ThaoGoTriGiacSaiLam-01.mp3
http://kinh.kinhphat.org/kinhgiang/NhatHanh/2007/03-02-07-ThaoGoTriGiacS

hyperlink: ../
path: ..
cur_link: http://kinh.kinhphat.org/kinhgiang/NhatHanh/BonLoaiXucThuc/
http://kinh.kinhphat.org/kinhgiang/NhatHanh/BonLoaiXucThuc/../
http://kinh.kinhphat.org/kinhgiang/NhatHanh/BonLoaiXucThuc/4%20loai%20Xuc%20thuc_01.mp3
http://kinh.kinhphat.org/kinhgiang/NhatHanh/BonLoaiXucThuc/4%20loai%20Xuc%20thuc_02.mp3
http://kinh.kinhphat.org/kinhgiang/NhatHanh/BonLoaiXucThuc/4%20loai%20Xuc%20thuc_03.mp3
http://kinh.kinhphat.org/kinhgiang/NhatHanh/BonLoaiXucThuc/4%20loai%20Xuc%20thuc_04.mp3
http://kinh.kinhphat.org/kinhgiang/NhatHanh/BonLoaiXucThuc/4%20loai%20Xuc%20thuc_05.mp3
http://kinh.kinhphat.org/kinhgiang/NhatHanh/BonLoaiXucThuc/4%20loai%20Xuc%20thuc_06.mp3
http://kinh.kinhphat.org/kinhgiang/NhatHanh/BonLoaiXucThuc/4%20loai%20Xuc%20thuc_07.mp3
http://kinh.kinhphat.org/kinhgiang/NhatHanh/BonLoaiXucThuc/4%20loai%20Xuc%20thuc_08.mp3
http://kinh.kinhphat.org/kinhgiang/NhatHanh/BonLoaiXucThuc/4%20loai%20Xuc%20thuc_09.mp3
http://kinh.kinhphat.org/kinhgiang/NhatHanh/BonLoai

hyperlink: ../
path: ..
cur_link: http://kinh.kinhphat.org/kinhgiang/NhatHanh/TrinhCongSon/
http://kinh.kinhphat.org/kinhgiang/NhatHanh/TrinhCongSon/../
http://kinh.kinhphat.org/kinhgiang/NhatHanh/TrinhCongSon/TrinhCongSon-1.mp3
http://kinh.kinhphat.org/kinhgiang/NhatHanh/TrinhCongSon/TrinhCongSon-2.mp3
http://kinh.kinhphat.org/kinhgiang/NhatHanh/TrinhCongSon/TrinhCongSon-3.mp3
hyperlink: ../
path: ..
cur_link: http://kinh.kinhphat.org/kinhgiang/NhatHanh/TuyenTap1/
http://kinh.kinhphat.org/kinhgiang/NhatHanh/TuyenTap1/../
http://kinh.kinhphat.org/kinhgiang/NhatHanh/TuyenTap1/CauHoiVaTraLoi-02-08-02-1.mp3
http://kinh.kinhphat.org/kinhgiang/NhatHanh/TuyenTap1/CauHoiVaTraLoi-02-08-02-2.mp3
http://kinh.kinhphat.org/kinhgiang/NhatHanh/TuyenTap1/ChonLuaHonNhan-04-08-98.mp3
http://kinh.kinhphat.org/kinhgiang/NhatHanh/TuyenTap1/ConNguoiLyTuongVaHanhPhuc1.mp3
http://kinh.kinhphat.org/kinhgiang/NhatHanh/TuyenTap1/ConNguoiLyTuongVaHanhPhuc2.mp3
http://kinh.kinhphat.org/kinhgiang/NhatHanh/TuyenTap

hyperlink: ../
path: ..
cur_link: http://kinh.kinhphat.org/kinhgiang/NhatHanh/TuyenTap5/
http://kinh.kinhphat.org/kinhgiang/NhatHanh/TuyenTap5/../
http://kinh.kinhphat.org/kinhgiang/NhatHanh/TuyenTap5/Hanh%20phuc%20trong%20khi%20tu_01.mp3
http://kinh.kinhphat.org/kinhgiang/NhatHanh/TuyenTap5/Khoa%20tu%20cho%20nguoi%20Xuat%20gia%20A_01.mp3
http://kinh.kinhphat.org/kinhgiang/NhatHanh/TuyenTap5/Khoa%20tu%20cho%20nguoi%20Xuat%20gia%20A_02.mp3
http://kinh.kinhphat.org/kinhgiang/NhatHanh/TuyenTap5/Khoa%20tu%20cho%20nguoi%20Xuat%20gia%20A_03.mp3
http://kinh.kinhphat.org/kinhgiang/NhatHanh/TuyenTap5/Lam%20lai%20tham%20tinh_01.mp3
http://kinh.kinhphat.org/kinhgiang/NhatHanh/TuyenTap5/Lua%20bo%20de%20tam%20con%20chay_01.mp3
http://kinh.kinhphat.org/kinhgiang/NhatHanh/TuyenTap5/Lua%20bo%20de%20tam%20con%20chay_02.mp3
http://kinh.kinhphat.org/kinhgiang/NhatHanh/TuyenTap5/LyTuongVaTinhThuong-a.mp3
http://kinh.kinhphat.org/kinhgiang/NhatHanh/TuyenTap5/LyTuongVaTinhThuong-b.mp3
http://kinh.kinhphat.o

hyperlink: ../
path: ..
cur_link: http://kinh.kinhphat.org/kinhgiang/NhatHanh/2003/ChatBoAoAnh/
http://kinh.kinhphat.org/kinhgiang/NhatHanh/2003/ChatBoAoAnh/../
http://kinh.kinhphat.org/kinhgiang/NhatHanh/2003/ChatBoAoAnh/01-TNH-2003-12-21-VN.mp3
http://kinh.kinhphat.org/kinhgiang/NhatHanh/2003/ChatBoAoAnh/02-TNH-2003-12-21-VN.mp3
http://kinh.kinhphat.org/kinhgiang/NhatHanh/2003/ChatBoAoAnh/03-TNH-2003-12-21-VN.mp3
http://kinh.kinhphat.org/kinhgiang/NhatHanh/2003/ChatBoAoAnh/04-TNH-2003-12-21-VN.mp3
http://kinh.kinhphat.org/kinhgiang/NhatHanh/2003/ChatBoAoAnh/05-TNH-2003-12-21-VN.mp3
http://kinh.kinhphat.org/kinhgiang/NhatHanh/2003/ChatBoAoAnh/06-TNH-2003-12-21-VN.mp3
http://kinh.kinhphat.org/kinhgiang/NhatHanh/2003/ChatBoAoAnh/07-TNH-2003-12-21-VN.mp3
http://kinh.kinhphat.org/kinhgiang/NhatHanh/2003/ChatBoAoAnh/08-TNH-2003-12-21-VN.mp3
hyperlink: ../
path: ..
cur_link: http://kinh.kinhphat.org/kinhgiang/NhatHanh/2003/DanhTuKienGiai/
http://kinh.kinhphat.org/kinhgiang/NhatHanh/2003/Dan

hyperlink: ../
path: ..
cur_link: http://kinh.kinhphat.org/kinhgiang/NhatHanh/2004/DoiTriCanCo2/
http://kinh.kinhphat.org/kinhgiang/NhatHanh/2004/DoiTriCanCo2/../
http://kinh.kinhphat.org/kinhgiang/NhatHanh/2004/DoiTriCanCo2/01%20TNH-2004-02-04-VN.mp3
http://kinh.kinhphat.org/kinhgiang/NhatHanh/2004/DoiTriCanCo2/02%20TNH-2004-02-04-VN.mp3
http://kinh.kinhphat.org/kinhgiang/NhatHanh/2004/DoiTriCanCo2/03%20TNH-2004-02-04-VN.mp3
http://kinh.kinhphat.org/kinhgiang/NhatHanh/2004/DoiTriCanCo2/04%20TNH-2004-02-04-VN.mp3
http://kinh.kinhphat.org/kinhgiang/NhatHanh/2004/DoiTriCanCo2/05%20TNH-2004-02-04-VN.mp3
http://kinh.kinhphat.org/kinhgiang/NhatHanh/2004/DoiTriCanCo2/06%20TNH-2004-02-04-VN.mp3
hyperlink: ../
path: ..
cur_link: http://kinh.kinhphat.org/kinhgiang/NhatHanh/2004/DoiTriCanCo3/
http://kinh.kinhphat.org/kinhgiang/NhatHanh/2004/DoiTriCanCo3/../
http://kinh.kinhphat.org/kinhgiang/NhatHanh/2004/DoiTriCanCo3/01%20TNH-2004-02-18-VN.mp3
http://kinh.kinhphat.org/kinhgiang/NhatHanh/2004/Do

In [17]:
f = open("all_path_site.txt", "w")
for site_path in all_links:
    f.write(site_path + '\n');
f.close()


In [16]:
# this function will list all links as posible of a website
link = 'http://hdgsnn.gov.vn/files/'
#link = "http://1506.unknownsecret.info/mp3/Michael+Learns+to+Rock/"
print ("Geting infomation from: " + link)

all_links = []
all_links.append(link)
for cur_link in all_links:
    # check if current link is a file. 
    if cur_link[-1] == '/':   # process if current link is folder, skip if it's a file
        soup = get_web_data_from_link(cur_link);
        if soup != -1:  # handle if data is responded
            hyperlink_tags = soup.find_all('a')
            for hyperlinks in hyperlink_tags:
                hyperlink = hyperlinks.get('href')
                if hyperlink.find('http') != -1:  # full link
                    downlink = hyperlink
                    print ("full link: " + downlink)
                elif hyperlink.find('../') != -1:  # parents
                    
                    downlink = cur_link.rsplit('/', 2)[0]
                    downlink = downlink + '/'
                    print ("parents: " + downlink)
                else:
                    downlink = cur_link + hyperlink
                    print ("downlink: " + downlink)
                
                #if hyperlink.find('/') != -1:     # if hyperlink contains /  then check duplicate in path
                #    path = hyperlink.rsplit('/', 1)[0]            # get the first part from last /
                #    if cur_link.find(path) != -1:      # found overlap between links and hyperlink
                #                                    # then remove overlap
                #        downlink = downlink.replace(path, '',1)
                #        print ("downlink overlap: " + downlink)
                if downlink not in all_links: # not exist in list, then add into it
                    all_links.append(downlink)
                    print("Added: " + downlink)

Geting infomation from: http://hdgsnn.gov.vn/files/
downlink: http://hdgsnn.gov.vn/files//
Added: http://hdgsnn.gov.vn/files//
downlink: http://hdgsnn.gov.vn/files/anhbaiviet/
Added: http://hdgsnn.gov.vn/files/anhbaiviet/
downlink: http://hdgsnn.gov.vn/files/anhbanner/
Added: http://hdgsnn.gov.vn/files/anhbanner/
downlink: http://hdgsnn.gov.vn/files/anhhosocanbo/
Added: http://hdgsnn.gov.vn/files/anhhosocanbo/
downlink: http://hdgsnn.gov.vn/files/anhloaitin/
Added: http://hdgsnn.gov.vn/files/anhloaitin/
downlink: http://hdgsnn.gov.vn/files/anhtintuc/
Added: http://hdgsnn.gov.vn/files/anhtintuc/
downlink: http://hdgsnn.gov.vn/files/gocanh/
Added: http://hdgsnn.gov.vn/files/gocanh/
downlink: http://hdgsnn.gov.vn/files/photos/
Added: http://hdgsnn.gov.vn/files/photos/
downlink: http://hdgsnn.gov.vn/files///files/
Added: http://hdgsnn.gov.vn/files///files/
downlink: http://hdgsnn.gov.vn/files//anhbaiviet/
Added: http://hdgsnn.gov.vn/files//anhbaiviet/
downlink: http://hdgsnn.gov.vn/files//

downlink: http://hdgsnn.gov.vn/files/gocanh//files/
Added: http://hdgsnn.gov.vn/files/gocanh//files/
downlink: http://hdgsnn.gov.vn/files/gocanh/anh/
Added: http://hdgsnn.gov.vn/files/gocanh/anh/
downlink: http://hdgsnn.gov.vn/files/photos//files/
Added: http://hdgsnn.gov.vn/files/photos//files/
downlink: http://hdgsnn.gov.vn/files/photos/8/
Added: http://hdgsnn.gov.vn/files/photos/8/
downlink: http://hdgsnn.gov.vn/files/photos/chao-don-tan-sinh-k22/
Added: http://hdgsnn.gov.vn/files/photos/chao-don-tan-sinh-k22/
downlink: http://hdgsnn.gov.vn/files/photos/gocanh/
Added: http://hdgsnn.gov.vn/files/photos/gocanh/
object of type 'NoneType' has no len()
downlink: http://hdgsnn.gov.vn/files//anhbaiviet//files//
Added: http://hdgsnn.gov.vn/files//anhbaiviet//files//
downlink: http://hdgsnn.gov.vn/files//anhbaiviet/Images/
Added: http://hdgsnn.gov.vn/files//anhbaiviet/Images/
downlink: http://hdgsnn.gov.vn/files//anhbaiviet/_thumbs/
Added: http://hdgsnn.gov.vn/files//anhbaiviet/_thumbs/
down

Added: http://hdgsnn.gov.vn/files//anhtintuc/95.jpg
downlink: http://hdgsnn.gov.vn/files//anhtintuc/96.jpg
Added: http://hdgsnn.gov.vn/files//anhtintuc/96.jpg
downlink: http://hdgsnn.gov.vn/files//anhtintuc/97.jpg
Added: http://hdgsnn.gov.vn/files//anhtintuc/97.jpg
downlink: http://hdgsnn.gov.vn/files//anhtintuc/98.jpg
Added: http://hdgsnn.gov.vn/files//anhtintuc/98.jpg
downlink: http://hdgsnn.gov.vn/files//anhtintuc/99.jpg
Added: http://hdgsnn.gov.vn/files//anhtintuc/99.jpg
downlink: http://hdgsnn.gov.vn/files//anhtintuc/logovien.jpg
Added: http://hdgsnn.gov.vn/files//anhtintuc/logovien.jpg
downlink: http://hdgsnn.gov.vn/files//anhtintuc/quochuy.png
Added: http://hdgsnn.gov.vn/files//anhtintuc/quochuy.png
downlink: http://hdgsnn.gov.vn/files//gocanh//files//
Added: http://hdgsnn.gov.vn/files//gocanh//files//
downlink: http://hdgsnn.gov.vn/files//gocanh/anh/
Added: http://hdgsnn.gov.vn/files//gocanh/anh/
downlink: http://hdgsnn.gov.vn/files//photos//files//
Added: http://hdgsnn.gov.vn/

downlink: http://hdgsnn.gov.vn/files/anhbaiviet/Images//files/anhbaiviet/
Added: http://hdgsnn.gov.vn/files/anhbaiviet/Images//files/anhbaiviet/
downlink: http://hdgsnn.gov.vn/files/anhbaiviet/Images/0803201804.jpg
Added: http://hdgsnn.gov.vn/files/anhbaiviet/Images/0803201804.jpg
downlink: http://hdgsnn.gov.vn/files/anhbaiviet/Images/0831.jpg
Added: http://hdgsnn.gov.vn/files/anhbaiviet/Images/0831.jpg
downlink: http://hdgsnn.gov.vn/files/anhbaiviet/Images/0832.jpg
Added: http://hdgsnn.gov.vn/files/anhbaiviet/Images/0832.jpg
downlink: http://hdgsnn.gov.vn/files/anhbaiviet/Images/0834.jpg
Added: http://hdgsnn.gov.vn/files/anhbaiviet/Images/0834.jpg
downlink: http://hdgsnn.gov.vn/files/anhbaiviet/Images/0835.jpg
Added: http://hdgsnn.gov.vn/files/anhbaiviet/Images/0835.jpg
downlink: http://hdgsnn.gov.vn/files/anhbaiviet/Images/0836.jpg
Added: http://hdgsnn.gov.vn/files/anhbaiviet/Images/0836.jpg
downlink: http://hdgsnn.gov.vn/files/anhbaiviet/Images/0837.jpg
Added: http://hdgsnn.gov.vn/f

Added: http://hdgsnn.gov.vn/files/anhbaiviet/Images/hpk220162017.JPG
downlink: http://hdgsnn.gov.vn/files/anhbaiviet/Images/hstn.png
Added: http://hdgsnn.gov.vn/files/anhbaiviet/Images/hstn.png
downlink: http://hdgsnn.gov.vn/files/anhbaiviet/Images/ht1.JPG
Added: http://hdgsnn.gov.vn/files/anhbaiviet/Images/ht1.JPG
downlink: http://hdgsnn.gov.vn/files/anhbaiviet/Images/ht10.JPG
Added: http://hdgsnn.gov.vn/files/anhbaiviet/Images/ht10.JPG
downlink: http://hdgsnn.gov.vn/files/anhbaiviet/Images/ht11.JPG
Added: http://hdgsnn.gov.vn/files/anhbaiviet/Images/ht11.JPG
downlink: http://hdgsnn.gov.vn/files/anhbaiviet/Images/ht12.JPG
Added: http://hdgsnn.gov.vn/files/anhbaiviet/Images/ht12.JPG
downlink: http://hdgsnn.gov.vn/files/anhbaiviet/Images/ht13.JPG
Added: http://hdgsnn.gov.vn/files/anhbaiviet/Images/ht13.JPG
downlink: http://hdgsnn.gov.vn/files/anhbaiviet/Images/ht2.JPG
Added: http://hdgsnn.gov.vn/files/anhbaiviet/Images/ht2.JPG
downlink: http://hdgsnn.gov.vn/files/anhbaiviet/Images/ht3.J

...

downlink: http://hdgsnn.gov.vn/files//anhbaiviet/files/2020/LLKH%20Thanh%20vien%20cac%20HD%20nganh%202020/4-HD%20CNTT%20LLKH%202020//files//anhbaiviet/files/2020/LLKH%20Thanh%20vien%20cac%20HD%20nganh%202020/
Added: http://hdgsnn.gov.vn/files//anhbaiviet/files/2020/LLKH%20Thanh%20vien%20cac%20HD%20nganh%202020/4-HD%20CNTT%20LLKH%202020//files//anhbaiviet/files/2020/LLKH%20Thanh%20vien%20cac%20HD%20nganh%202020/
downlink: http://hdgsnn.gov.vn/files//anhbaiviet/files/2020/LLKH%20Thanh%20vien%20cac%20HD%20nganh%202020/4-HD%20CNTT%20LLKH%202020/4-HD%20CNTT-GS%20DQ%20A_0001.pdf
Added: http://hdgsnn.gov.vn/files//anhbaiviet/files/2020/LLKH%20Thanh%20vien%20cac%20HD%20nganh%202020/4-HD%20CNTT%20LLKH%202020/4-HD%20CNTT-GS%20DQ%20A_0001.pdf
downlink: http://hdgsnn.gov.vn/files//anhbaiviet/files/2020/LLKH%20Thanh%20vien%20cac%20HD%20nganh%202020/4-HD%20CNTT%20LLKH%202020/4-HD%20CNTT-GS%20Dinh%20Dung_0001.pdf
Added: http://hdgsnn.gov.vn/files//anhbaiviet/files/2020/LLKH%20Thanh%20vien%20cac%20HD%

downlink: http://hdgsnn.gov.vn/files//anhbaiviet/files/2020/LLKH%20Thanh%20vien%20cac%20HD%20nganh%202020/7-HD%20GTVT%20LLKH%202020//files//anhbaiviet/files/2020/LLKH%20Thanh%20vien%20cac%20HD%20nganh%202020/
Added: http://hdgsnn.gov.vn/files//anhbaiviet/files/2020/LLKH%20Thanh%20vien%20cac%20HD%20nganh%202020/7-HD%20GTVT%20LLKH%202020//files//anhbaiviet/files/2020/LLKH%20Thanh%20vien%20cac%20HD%20nganh%202020/
downlink: http://hdgsnn.gov.vn/files//anhbaiviet/files/2020/LLKH%20Thanh%20vien%20cac%20HD%20nganh%202020/7-HD%20GTVT%20LLKH%202020/7-HD%20GTVT-GS%20%20Pham%20Huy%20Khang.pdf
Added: http://hdgsnn.gov.vn/files//anhbaiviet/files/2020/LLKH%20Thanh%20vien%20cac%20HD%20nganh%202020/7-HD%20GTVT%20LLKH%202020/7-HD%20GTVT-GS%20%20Pham%20Huy%20Khang.pdf
downlink: http://hdgsnn.gov.vn/files//anhbaiviet/files/2020/LLKH%20Thanh%20vien%20cac%20HD%20nganh%202020/7-HD%20GTVT%20LLKH%202020/7-HD%20GTVT-GS%20PC%20Thang_0001.pdf
Added: http://hdgsnn.gov.vn/files//anhbaiviet/files/2020/LLKH%20Thanh

downlink: http://hdgsnn.gov.vn/files//anhbaiviet/files/2020/LLKH%20Thanh%20vien%20cac%20HD%20nganh%202020/File%20word%20DS%20TV%20cac%20HD%20nganh%202020//files//anhbaiviet/files/2020/LLKH%20Thanh%20vien%20cac%20HD%20nganh%202020/
Added: http://hdgsnn.gov.vn/files//anhbaiviet/files/2020/LLKH%20Thanh%20vien%20cac%20HD%20nganh%202020/File%20word%20DS%20TV%20cac%20HD%20nganh%202020//files//anhbaiviet/files/2020/LLKH%20Thanh%20vien%20cac%20HD%20nganh%202020/
downlink: http://hdgsnn.gov.vn/files//anhbaiviet/files/2020/LLKH%20Thanh%20vien%20cac%20HD%20nganh%202020/File%20word%20DS%20TV%20cac%20HD%20nganh%202020/DS%20TV%20HD%20nganh%20CNTT%202020.docx
Added: http://hdgsnn.gov.vn/files//anhbaiviet/files/2020/LLKH%20Thanh%20vien%20cac%20HD%20nganh%202020/File%20word%20DS%20TV%20cac%20HD%20nganh%202020/DS%20TV%20HD%20nganh%20CNTT%202020.docx
downlink: http://hdgsnn.gov.vn/files//anhbaiviet/files/2020/LLKH%20Thanh%20vien%20cac%20HD%20nganh%202020/File%20word%20DS%20TV%20cac%20HD%20nganh%202020/DS

object of type 'NoneType' has no len()
object of type 'NoneType' has no len()
object of type 'NoneType' has no len()
object of type 'NoneType' has no len()
object of type 'NoneType' has no len()
object of type 'NoneType' has no len()
object of type 'NoneType' has no len()
object of type 'NoneType' has no len()
object of type 'NoneType' has no len()
object of type 'NoneType' has no len()
object of type 'NoneType' has no len()
object of type 'NoneType' has no len()
object of type 'NoneType' has no len()
object of type 'NoneType' has no len()
object of type 'NoneType' has no len()
object of type 'NoneType' has no len()
object of type 'NoneType' has no len()
object of type 'NoneType' has no len()
object of type 'NoneType' has no len()
object of type 'NoneType' has no len()
object of type 'NoneType' has no len()
object of type 'NoneType' has no len()
object of type 'NoneType' has no len()
object of type 'NoneType' has no len()
object of type 'NoneType' has no len()
object of type 'NoneType'

downlink: http://hdgsnn.gov.vn/files//anhbaiviet/_thumbs/Files/2020/BDK%2021%20HD%20nganh%202020/HD%20VH-NT-TT//files//anhbaiviet/_thumbs/Files/2020/BDK%2021%20HD%20nganh%202020/
Added: http://hdgsnn.gov.vn/files//anhbaiviet/_thumbs/Files/2020/BDK%2021%20HD%20nganh%202020/HD%20VH-NT-TT//files//anhbaiviet/_thumbs/Files/2020/BDK%2021%20HD%20nganh%202020/
object of type 'NoneType' has no len()
object of type 'NoneType' has no len()
object of type 'NoneType' has no len()
object of type 'NoneType' has no len()
object of type 'NoneType' has no len()
object of type 'NoneType' has no len()
object of type 'NoneType' has no len()
object of type 'NoneType' has no len()
object of type 'NoneType' has no len()
object of type 'NoneType' has no len()
object of type 'NoneType' has no len()
object of type 'NoneType' has no len()
object of type 'NoneType' has no len()
object of type 'NoneType' has no len()
object of type 'NoneType' has no len()
object of type 'NoneType' has no len()
object of type 'NoneT